# LPI Tutorial

Get mean LPI for each REW channel

In [99]:
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
import gdal
import os
import datetime
from datetime import timedelta
import numpy as np
import fiona
import shapely
from shapely.geometry import Point
from os.path import dirname
import glob
import rasterio
import sys
import pickle
from functools import partial
import geopandas as gp

parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','lib'))


# Add LPI to streams shapefile


In [102]:
streams_file = glob.glob(os.path.join(parent_dir, 'raw_data','streams_poly','*.shp'))[0]
streams = gp.read_file(streams_file)
geos = streams.geometry.values
ids = streams.stream.values
pts_list = []
ids_list = []
for i,geo in enumerate(geos):
    pts = [pair for pair in geo.coords]
    pts_list = pts_list + pts
    ids_list = ids_list + len(pts)*[ids[i]]
src = rasterio.open('/Users/daviddralle/Dropbox/research/LPI_project/data/lpi_predicted.tif')

samples = np.array([item[0] for item in src.sample(pts_list)])
samples[samples<0] = 0
samples[samples>100]=100
tempdf = pd.DataFrame.from_dict({'lpi':samples, 'stream':ids_list})
new_streams = pd.DataFrame(tempdf.groupby('stream')['lpi'].mean()).merge(streams, left_on='stream', right_on='stream')
new_streams = gp.GeoDataFrame(new_streams)
new_streams.to_file(streams_file)

In [4]:
lpi_files = glob.glob(os.path.join('/Volumes/zee_data/lpi','*.tif'))
points_file = glob.glob(os.path.join(parent_dir, 'raw_data','streams_points','*.shp'))[0]
points = gp.read_file(points_file)

mean_lpis = {i:0 for i in basins.cat}
for raster_file in lpi_files:
    lpis_dict = {i:[] for i in basins.cat}
    filename = raster_file.split('/')[-1]
    mind = filename.index('m')
    month = int(filename[(mind+1):(mind+3)])
    # get raster as numpy array
    gdata = gdal.Open(raster_file)
    gt = gdata.GetGeoTransform()
    data = gdata.ReadAsArray().astype(np.float)
    
    # loop over basins, get points within each basin
    for basinindex, basinrow in basins.iterrows():
        for index, row in points.iterrows():
            if row.geometry.within(basinrow.geometry):
                pos = (row.geometry.x, row.geometry.y)
                x = int((pos[0] - gt[0])/gt[1])
                y = int((pos[1] - gt[3])/gt[5])
                lpis_dict[basinrow['cat']].append(data[x,y])
    
    lpis_means = {i:np.mean(lpis_dict[i]) for i in lpis_dict.keys()}
    for basin_index in lpis_means.keys():
        mean_lpis[basin_index] = mean_lpis[basin_index] + lpis_means[basin_index]/12.0
        
pickle.dump(mean_lpis, open(os.path.join(parent_dir, 'raw_data', 'mean_rew_lpis.p'), 'wb'))